# TORCH.AUTOGRAD를 사용한 자동 미분
- 신경망을 학습할 때 가장 자주 사용되는 알고리즘은 **역전파**이다.
- 이 알고리즘에서, 매개변수(모델 가중치)는 주어진 매개변수에 대한 손실 함수의 변화도(gradient)에 따라 조정된다.

- 이러한 변화도를 계산하기 위해, Pytorch에서 `torch.autograd` 라고 불리는 자동 미분 엔진이 내장되어 있다.
- 이는 모든 계산 그래프에 대한 변화도의 자동 계산을 지원한다.

- 입력 x, 매개변수 `w`와 `b`, 그리고 일부 손실 함수가 있는 가장 간단한 단일 계층 신경망을 만들어보자.

In [7]:
import torch

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad= True)
b = torch.randn(3, requires_grad= True)
z = torch.matmul(x,w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

![텐서, 함수와 연산그래프](https://tutorials.pytorch.kr/_images/comp-graph.png)
- 이 신경망에서, `w`와 `b`는 최적화를 해야하는 변수이다.
- 이러한 변수들에 대한 손실함수의 변화도를 계산해야한다.
- 이를 해당 텐서에 `required_grad` 속성을 이용하자.
- 참고: `required_grad`의 값은 텐서를 생성할 때 설정하거나, `x.required_grad_(True)` 메서드를 사용하여 나중에 설정할 수도 있다.

연산 그래프를 구성하기 위해 텐서에 적용하는 함수는 사실 `Function` 클래스의 객체이다. 이 객체는 *순전파* 방향으로 함수를 계산하는 방법과, *역전파* 단계에서 도함수(derivative)를 계산할 수 있다.   
역전파 함수에 대한 참조는 텐서의 `grad_fn` 속성에 저장된다.

In [4]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x00000265B2624430>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x00000265B36D4CD0>


## 변화도(Gradient) 계산기
- 신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 도함수(derivate)를 계산해야한다.
- x와 y의 일부 고정값에서 loss의 w에대한 도함수 및 loss의 b에 대한 도함수가 필요하다.
- 계산을 위해, `loss.backward()` 를 호출한 다음 `w.grad`와 `b.grad`에서 값을 가져온다.

In [8]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2196, 0.0671, 0.3310],
        [0.2196, 0.0671, 0.3310],
        [0.2196, 0.0671, 0.3310],
        [0.2196, 0.0671, 0.3310],
        [0.2196, 0.0671, 0.3310]])
tensor([0.2196, 0.0671, 0.3310])


- 참고
    1. 연산 그래프의 잎(leaf) 노드들 중 `required_grad` 속성이 `True`로 설정된 노드들의 `grad`속성만 구할 수 있다. 그래프의 다른 모든 노드에서는 변화도가 유효하지 않는다.
    2. 성능 상의 이유로, 주어진 그래프에서의 `backward`를 사용한 변화도 계산은 한 번만 수행할 수 있다. 만약 동일한 그래프에서 여러번의 `backward` 호출이 필요하면, `backward`호출 시 `retrain_grad =True`를 전달해야 한다.

## 변화도 추적 멈추기
- 기본적으로 `required_grad = True` 인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원한다.
- 하지만 모델을 학습한 뒤, 입력 데이터를 단순히 적용하기만 하는 경우와 같이, *순전파* 연산만 필요한 경우엔 추적이나 지원이 필요없다.
- 연산코드를 `torch.no_grid()` 블록으로 둘러싸서 연산 및 추적을 멈출 수 있다.
- 동일하게, 텐서에 `detch()` 메서드를 사용하는 방법도 있다.

In [11]:
# 1
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

# 2
z = torch.matmul(x, w) + b
z_detch = z.detach()
print(z_detch.requires_grad)

True
False
False


### 변화도 추적을 멈춰야 하는 경우
1. 신경망 일부 매개변수를 고정된 매개변수(frozen parameter)로 표시한다.
2. 변화도를 추적하지 않는 텐서의 연산이 더 효율적이므로, 순전파에서는 추적하지 않고 속도가 빠르다.

## 연산 그래프에 대한 추가 정보
- 개념적으로 autograd는 데이터(tensor) 및 실행된 모든 연산들의 기록을 Fuction 객체로 구성된 방향성 비순환 그래프(DAG; Directed Acyclic Graph)에 저장합니다.
- 이 방향성 비순환 그래프(DAG)의 잎(leave)은 입력 텐서이고, 뿌리(root)는 결과 텐서이다.
- 이 그래프를 뿌리에서 잎까지 추적하면 연쇄 법칙(chain rule)에 따라 변화도를 자동으로 계산할 수 있다.   

순전파 단계에서, `autograd`는 다음 두 가지 작업을 동시에 수행한다.
1. 요청된 연산을 수행하여 결과 텐서를 계산
2. DAG에 연산의 변화도 기능(gradient function)을 유지   

역전파 단계에서, DAG 뿌리(root)에서 `.backward()` 가 호출될 때 시작됩니다. `autograd` 는   
- 각 `.grad_fn` 으로부터 변화도 계산
- 각 텐서의 `.grad` 속성에 계산 결과를 쌓고(accumulate),
- 연쇄 법칙을 사용하여, 모든 잎(leaf) 텐서들까지 전파(propagate)한다.

Pytorch에서 DAG들은 동적이다.   
그래프가 처음부터 다시 생성된다. 매번 `.backward()`가 호출되고 나면, `autograd`는 새로운 그래프를 채운다.(populate)   
이러한 점으로 흐름 제어 구문들을 사용할 수 있다. 매번 반복이 필요할 때 마다 필요하면 모양이나 크기, 연산을 바꿀 수 있다는 장점이있다.